In [1]:
import numpy as np
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Dense
from keras.layers.merge import concatenate

In [3]:
in_seq1 = np.arange(10, 100, 10)
in_seq2 = np.arange(15, 105, 10)
out_seq = np.array([in_seq1[i]+in_seq2[i] for i in range(in_seq1.shape[0])])
in_seq1 = in_seq1.reshape(in_seq1.shape[0], 1)
in_seq2 = in_seq2.reshape(in_seq2.shape[0], 1)
out_seq = out_seq.reshape(out_seq.shape[0], 1)
# horizontally stack columns
dataset  = np.hstack((in_seq1, in_seq2, out_seq))
print(dataset.shape)
print(dataset)

(9, 3)
[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]


In [6]:
def split_sequence(series, n_step):
    x_list, y_list = [], [] 
    for s in range(series.shape[0]-n_step):
        x_list.append(series[s:s+n_step])
        y_list.append(series[s+n_step])
    X, y = np.array(x_list), np.array(y_list)
    return X, y

In [8]:
n_step = 3
X, y = split_sequence(dataset, n_step)
for i in range(X.shape[0]):
    print(X[i], y[i])

[[10 15 25]
 [20 25 45]
 [30 35 65]] [40 45 85]
[[20 25 45]
 [30 35 65]
 [40 45 85]] [ 50  55 105]
[[ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]] [ 60  65 125]
[[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]] [ 70  75 145]
[[ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]] [ 80  85 165]
[[ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]] [ 90  95 185]


In [9]:
# flaten output
resulting_feature = X.shape[1] * X.shape[2]
X_flat = X.reshape(X.shape[0], resulting_feature)
X_flat

array([[ 10,  15,  25,  20,  25,  45,  30,  35,  65],
       [ 20,  25,  45,  30,  35,  65,  40,  45,  85],
       [ 30,  35,  65,  40,  45,  85,  50,  55, 105],
       [ 40,  45,  85,  50,  55, 105,  60,  65, 125],
       [ 50,  55, 105,  60,  65, 125,  70,  75, 145],
       [ 60,  65, 125,  70,  75, 145,  80,  85, 165]])

In [10]:
resulting_output = y.shape[1]

In [13]:
# define model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=resulting_feature))
model.add(Dense(resulting_output))
model.compile(optimizer='adam', loss='mse')

In [14]:
model.fit(X_flat, y, epochs=2000, verbose=0)

Instructions for updating:
Use tf.cast instead.


In [15]:
x_test_raw = np.array([[70, 75, 145], [80, 85, 165], [90, 95, 185]])
x_test = x_test_raw.reshape(1, resulting_feature)
x_test

array([[ 70,  75, 145,  80,  85, 165,  90,  95, 185]])

In [16]:
yhat = model.predict(x_test)
print(yhat)

[[100.03339 104.93609 205.65616]]


## Multi-output MLP Model

In [17]:
# define model
visible = Input(shape=(resulting_feature, ))
dense = Dense(100, activation='relu')(visible)

In [18]:
#define output
output1 = Dense(1)(dense)
output2 = Dense(1)(dense)
output3 = Dense(1)(dense)
#tie together
model = Model(inputs=visible, outputs=[output1, output2, output3])
model.compile(optimizer='adam', loss='mse')

In [19]:
print(y)

[[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]


In [22]:
y1 = y[:, 0].reshape((y.shape[0], 1))
y2 = y[:, 1].reshape((y.shape[0], 1))
y3 = y[:, 2].reshape((y.shape[0], 1))
print(y1, y2, y3)

[[40]
 [50]
 [60]
 [70]
 [80]
 [90]] [[45]
 [55]
 [65]
 [75]
 [85]
 [95]] [[ 85]
 [105]
 [125]
 [145]
 [165]
 [185]]


In [24]:
model.fit(X_flat, [y1, y2, y3], epochs=2000, verbose=0)

In [29]:
yhat = model.predict(x_test)
print(yhat)

[array([[100.00844]], dtype=float32), array([[105.00436]], dtype=float32), array([[206.55031]], dtype=float32)]


In [27]:
X_flat

array([[ 10,  15,  25,  20,  25,  45,  30,  35,  65],
       [ 20,  25,  45,  30,  35,  65,  40,  45,  85],
       [ 30,  35,  65,  40,  45,  85,  50,  55, 105],
       [ 40,  45,  85,  50,  55, 105,  60,  65, 125],
       [ 50,  55, 105,  60,  65, 125,  70,  75, 145],
       [ 60,  65, 125,  70,  75, 145,  80,  85, 165]])